In [1]:
import pandas as pd
import numpy as np

## A side note about value_counts

`value_counts()` is a useful function that shows you how many unique values are contained inside a Dataframe or Series. Here is what using it looks like

In [2]:
# This is a dataset containing information about employees at a company. We
# can see that each employee name is unique, but the groups they belong to are
# not. The three groups are accounting, engineering, and HR.
df = pd.read_csv("https://raw.githubusercontent.com/CUNY-CISC-3225/datasets/main/company/groups.csv")
df

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Frank,HR
5,Juan,Engineering
6,Alice,Engineering
7,Steve,Accounting


In [3]:
# value_counts() can answer the question "How many people are in each group?"
# (Or, more specifically, how many unique values are there in the "group" column?
# And how many times do each of those values appear?)
df['group'].value_counts()

group
Engineering    4
Accounting     2
HR             2
Name: count, dtype: int64

In [4]:
# We can apply value_counts to a DataFrame, but (at least in this case), the results
# are not as useful. We see the Series object has been constructed with a multi-index,
# and each employee-group pair only occurs once.
df.value_counts()

employee  group      
Alice     Engineering    1
Bob       Accounting     1
Frank     HR             1
Jake      Engineering    1
Juan      Engineering    1
Lisa      Engineering    1
Steve     Accounting     1
Sue       HR             1
Name: count, dtype: int64

# Concatenation

Sometimes, your data will not come from the same place. It is common for large datasets to be split among multiple files, and to work with the data, you must combine them together after loading.


In [5]:
# We are familiar with Python list concatenation already. With Python lists, the
# plus (+) operator indicates concatenation, and we can use it to combine
# the contents of multiple lists into a single list.

a = [1, 2, 3]
b = [100, 200, 300]
c = [1000, 2000, 3000]

a + b + c

[1, 2, 3, 100, 200, 300, 1000, 2000, 3000]

In [6]:
# Similarly, NumPy will concatenate for us. But because + indicates addition when
# used with NumPy arrays, we have to call its concatenate() function:
np.concatenate([a, b, c])

array([   1,    2,    3,  100,  200,  300, 1000, 2000, 3000])

In [7]:
# If we're dealing with lists of lists, Python concatenation is
# pretty simple: it does the equivalent of concatenation along the 0th axis
# in NumPy. We cannot change this behavior.
x = [
    [1, 2],
    [3, 4]
]

y = [
    [100, 200],
    [300, 400]
]

x + y

[[1, 2], [3, 4], [100, 200], [300, 400]]

In [8]:
# With NumPy, we can use the `axis` parameter to control how the concatenation
# is performed. For example, we can perform concatenation along the 0th axis
# (the default), and the result is similar to what we saw above:
np.concatenate([x, y], axis=0)

array([[  1,   2],
       [  3,   4],
       [100, 200],
       [300, 400]])

In [9]:
# Alternatively, we can perform concatenation along the 1st axis,
# and as a result, the second array is placed to the right of the first instead
# of underneath:
np.concatenate([x, y], axis=1)

array([[  1,   2, 100, 200],
       [  3,   4, 300, 400]])

In [10]:
# Concatenating with Series objects is, on the surface, pretty straightforward.
# Since Series are like one-dimensional lists, there is only one axis over
# which the concatenation can be performed.
#
# Unlike NumPy arrays, Series objects have indices. What happens to the indices
# when two Series with the same index are concatenated? We see in the result
# that the indices have duplicates! This is allowed in Pandas, and while it may
# be acceptable in some situations, it can sometimes lead to unexpected results.
s1 = pd.Series(['A', 'B', 'C'])
s2 = pd.Series(['D', 'E', 'F'])

print(s1)
print()
print(s2)
print()
pd.concat([s1, s2])

0    A
1    B
2    C
dtype: object

0    D
1    E
2    F
dtype: object



0    A
1    B
2    C
0    D
1    E
2    F
dtype: object

In [11]:
# For example, if we try to find values based on their index, we will get multiple
# values:
pd.concat([s1, s2]).loc[0]

0    A
0    D
dtype: object

In [26]:
pd.concat([s1, s2])

0    A
1    B
2    C
0    D
1    E
2    F
dtype: object

In [12]:
# If we want to detect and avoid this situation, we can pass the `verify_integrity`
# argument into the concat() function. It will cause concat() to raise an
# exception if any index values overlap:
pd.concat([s1, s2], verify_integrity=True)

ValueError: Indexes have overlapping values: Index([0, 1, 2], dtype='int64')

In [ ]:
# Alternatively, we can call concat() with `ignore_index`, which will cause it
# to discard the indices of the incoming Series objects and replace them with a
# new, incrementing index starting from 0:
pd.concat([s1, s2], ignore_index=True)

0    A
1    B
2    C
3    D
4    E
5    F
dtype: object

In [ ]:
# DataFrames introduce additional complexity when concatenating.
# Let's create two new DataFrames with no overlapping row or column indices.
df1 = pd.DataFrame([[1, 2], [3, 4]], columns=['A', 'B'], index=[0, 1])
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=['C', 'D'], index=[2, 3])

display(df1)
display(df2)

,A,B
0,1,2
1,3,4


,C,D
2,5,6
3,7,8


In [ ]:
# When we concatenate them, we get a 4x4 output DataFrame with missing values!
# Why did this happen?
# The resulting dataframe necessarily must contain all the row indices in df1
# and df2, and all the columns in df1 and df2. df2 doesn't have a column A or B,
# and df1 doesn't have a column C or D. Similarly, df2 doesn't have a row 2 or 3,
# and df1 doesn't have a row 1 or 2. So there is nothing that Pandas can do but
# fill in the empty spaces with NaN:
pd.concat([df1, df2])

,A,B,C,D
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
2,NaN,NaN,5.0,6.0
3,NaN,NaN,7.0,8.0


In [ ]:
# Interestingly, we get the same result when performing column-wise concatenation:
pd.concat([df1, df2], axis=1)

,A,B,C,D
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
2,NaN,NaN,5.0,6.0
3,NaN,NaN,7.0,8.0


In [ ]:
# What if the DataFrames have overlapping rows?
df1 = pd.DataFrame([[1, 2], [3, 4]], columns=['A', 'B'], index=[0, 1])
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=['C', 'D'], index=[0, 1])

display(df1)
display(df2)

,A,B
0,1,2
1,3,4


,C,D
0,5,6
1,7,8


In [ ]:
# If we perform row-wise concatenation (which is the default and does not need to
# be specified), we get the same result.
pd.concat([df1, df2], axis=0)

,A,B,C,D
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
0,NaN,NaN,5.0,6.0
1,NaN,NaN,7.0,8.0


In [ ]:
# But if we perform column-wise concatenation, Pandas recognizes that df1
# and df2 have overlapping row indices. It assumes they represent the same
# data, and combines the columns together without creating null values:
pd.concat([df1, df2], axis=1)

,A,B,C,D
0,1,2,5,6
1,3,4,7,8


In [ ]:
# What if the columns, not rows, are overlapping?
df1 = pd.DataFrame([[1, 2], [3, 4]], columns=['A', 'B'], index=[0, 1])
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=['A', 'B'], index=[2, 3])

display(df1)
display(df2)

,A,B
0,1,2
1,3,4


,A,B
2,5,6
3,7,8


In [ ]:
# The opposite is true: row-wise concatenation does not create
# regions of missing values:
pd.concat([df1, df2], axis=0)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8


In [ ]:
# But column-wise concatenation does:
pd.concat([df1, df2], axis=1)

,A,B,A,B
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
2,NaN,NaN,5.0,6.0
3,NaN,NaN,7.0,8.0


# Merging

In some situations, you may be pulling data in from multiple datasets with different structures that would be beneficial to combine together. This illustrates how it is done with a realistic example, and covers the different types of merging.

In [27]:
# Here, we have several datasets describing employees at a small company.
# This dataset shows which employees belong to which group in the company.
groups_df = pd.read_csv("https://raw.githubusercontent.com/CUNY-CISC-3225/datasets/main/company/groups.csv")
groups_df

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Frank,HR
5,Juan,Engineering
6,Alice,Engineering
7,Steve,Accounting


In [28]:
# This dataset shows the year each employee was hired.
hired_df = pd.read_csv("https://raw.githubusercontent.com/CUNY-CISC-3225/datasets/main/company/hired_date.csv")
hired_df

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014
4,Matt,2001
5,Juan,2017
6,Alice,2019
7,Steve,2001


In [29]:
# This dataset shows which skills employees in each group are expected to have.
df_competencies = pd.read_csv("https://raw.githubusercontent.com/CUNY-CISC-3225/datasets/main/company/group_skills.csv")
df_competencies

,group_name,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [30]:
# Finally, this dataset shows who supervises each group.
supervisor_df = pd.read_csv("https://raw.githubusercontent.com/CUNY-CISC-3225/datasets/main/company/group_supervisors.csv")
supervisor_df

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [31]:
# Many companies are concerned with employee turnover. If a group has high turnover,
# it means that its employees quit often and have to be replaced. Groups with
# low turnover are able to keep employees for a long time.
#
# An easy way to determine turnover is to just compute the average hire date, and
# see how recent it is. We can determine this for the whole company pretty easily:
hired_df['hire_date'].mean()

2009.5

In [32]:
# But what if I want to know the turnover for a specific group, say engineering.
# I can't answer this with the DataFrames I have now, because the information
# is split across two DataFrames. I need to combine them together first.

In [33]:
# I can do this with the merge() function. If I call merge(), I have to specify
# a left DataFrame and a right DataFrame. So long as both DataFrames have a
# column in common, the merge() function with match rows on the left with
# rows on the right and combine them together.
#
# Recall that both groups_df and hired_df have an "employee" column. merge()
# will combine the employees in groups_df with their hire date in hired_df():
df_merged = pd.merge(groups_df, hired_df)
df_merged

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014
4,Juan,Engineering,2017
5,Alice,Engineering,2019
6,Steve,Accounting,2001


In [34]:
# Now we can answer the question. The average hire date of employees in
# engineering is 2013, which is a bit higher than the average hire date
# of the company as a whole. So we know that, on average, people in engineering
# were hired more recently than at the company as a whole.
df_merged.loc[df_merged['group'] == 'Engineering', 'hire_date'].mean()

2013.0

In [35]:
# But did the merge account for everyone? Go back and look at the groups_df
# and hired_df DataFrames above. Matt is listed in hired_df, and Frank is listed
# in groups_df. But neither appeared in the final merged output!

In [36]:
# The reason Matt and Frank aren't in the merged DataFrame is because Matt
# is in hired_df but not in groups_df. Similarly, Frank is in groups_df but not
# in hired_df. Because they aren't in both, the merge() function won't put them
# in the output DataFrame by default.
display(hired_df[hired_df['employee'] == 'Frank'])
print()
display(groups_df[groups_df['employee'] == 'Matt'])

,employee,hire_date


,employee,group


In [37]:
# The merge function performs what's called an inner merge by default. But there
# are several types of merges that handle this situation differently:

|Type|Operation|Example|
|----|---------|-------|
|Inner|Intersection|Left: ABCD<br />Right: CDE<br/>Result: CD|
|Outer|Union|Left: ABCD<br />Right: CDE<br/>Result: ABCDE|
|Left|Left set|Left: ABCD<br />Right: CDE<br/>Result: ABCD|
|Right|Right set|Left: ABCD<br />Right: CDE<br/>Result: CDE|

In [38]:
# We can use the `how` argument to control which merge type will be used.
# For example, by performing a left merge, the merge() function will use all
# employee in the left side (groups_df), even if they don't have a counterpart
# on the right. This causes Frank to appear with a NaN hire date.
#
# Try running this cell with different merge types to see how it affects the
# final output:
pd.merge(groups_df, hired_df, how='left')

,employee,group,hire_date
0,Bob,Accounting,2008.0
1,Jake,Engineering,2012.0
2,Lisa,Engineering,2004.0
3,Sue,HR,2014.0
4,Frank,HR,NaN
5,Juan,Engineering,2017.0
6,Alice,Engineering,2019.0
7,Steve,Accounting,2001.0


In [39]:
# Now let's try merging in supervisor data. Review supervisor_df above: each group
# supervisor appears only once in the dataset. Why do the same supervisors appear
# multiple times in the merged DataFrame below?
df_merged_supervisor = pd.merge(df_merged, supervisor_df)
df_merged_supervisor

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve
4,Juan,Engineering,2017,Guido
5,Alice,Engineering,2019,Guido
6,Steve,Accounting,2001,Carly


In [40]:
# Our original merge (groups and hire dates) is fundamentally a different type
# of merge than the second (groups and supervisors). Below is a review of three
# different types of merges:

# One-to-one merges
Merging the hire dates and groups is a *one-to-one merge*. For every merge key, there is one row in the left DataFrame associated with one row in the right DataFrame (missing rows are OK).

One-to-one merges look like this:

Left:

|key|value_left|
|---|-----|
|a|1|
|b|2|
|c|2|

Right:

|key|value_right|
|---|-----|
|a|9|
|b|8|
|c|7|

Result:

|key|value_left|value_right|
|---|-----|-----|
|a|1|9|
|b|2|8|
|c|3|7|

# Many-to-one merges
Another type of merge is a *many-to-one merge*. In one side of the row, there is only one occurrence of each merge key. In the other side of the row, there are one or more occurrences of each merge key.

Many-to-one merges look like this:

Left (many):

|key|value_left|
|---|-----|
|a|1|
|a|2|
|b|3|
|c|4|

Right (one):

|key|value_right|
|---|-----|
|a|9|
|b|8|
|c|7|

Result:

|key|value_left|value_right|
|---|-----|-----|
|a|1|9|
|a|2|9|
|b|3|8|
|c|4|7|


# Many-to-many merges
In a *many-to-many* merge, the merge key can occur multiple times on either side. The result is the Cartesian (or cross) product of rows sharing a key.

Left (many):

|key|value_left|
|---|-----|
|a|1|
|a|2|
|b|3|
|c|4|

Right (one):

|key|value_right|
|---|-----|
|a|9|
|a|8|
|b|7|
|c|6|

Result:

|key|value_left|value_right|
|---|-----|-----|
|a|1|9|
|a|1|8|
|a|2|9|
|a|2|8|
|b|3|8|
|c|4|7|

In [41]:
# We now understand this is a many-to-one merge: there are many departments
# to one supervisor.
pd.merge(df_merged, supervisor_df)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve
4,Juan,Engineering,2017,Guido
5,Alice,Engineering,2019,Guido
6,Steve,Accounting,2001,Carly


In [42]:
# Finally, merging group competencies with our accumulated merged DataFrame is
# a many-to-many merge: several groups appear more than once on the left, and each
# group appears more than once on the right.
#
# Note that the left and right side have different names for the group column.
# On the left, it is "group", and on the right, it is "group_name". Pandas will
# not automatically recognize them, so we must use `left_on` and `right_on` arguments
# to specify the column names!
df_merged_final = pd.merge(df_merged_supervisor, df_competencies, left_on='group', right_on='group_name')
df_merged_final

,employee,group,hire_date,supervisor,group_name,skills
0,Bob,Accounting,2008,Carly,Accounting,math
1,Bob,Accounting,2008,Carly,Accounting,spreadsheets
2,Jake,Engineering,2012,Guido,Engineering,coding
3,Jake,Engineering,2012,Guido,Engineering,linux
4,Lisa,Engineering,2004,Guido,Engineering,coding
5,Lisa,Engineering,2004,Guido,Engineering,linux
6,Sue,HR,2014,Steve,HR,spreadsheets
7,Sue,HR,2014,Steve,HR,organization
8,Juan,Engineering,2017,Guido,Engineering,coding
9,Juan,Engineering,2017,Guido,Engineering,linux


In [43]:
# Here's an example query I can perform on this final merged DataFrame.
# Let's say I have a question about Microsoft Excel. Who should I ask?
df_merged_final[df_merged_final.skills == 'spreadsheets']

,employee,group,hire_date,supervisor,group_name,skills
1,Bob,Accounting,2008,Carly,Accounting,spreadsheets
6,Sue,HR,2014,Steve,HR,spreadsheets
13,Steve,Accounting,2001,Carly,Accounting,spreadsheets
